## Predicting Antibiotic Resistance in Salmonella enteriditis Using Isolate Data

## Introduction
> Antibiotic resistance is a pressing public health issue with widespread economic consequences. As bacterial pathogens like *Salmonella enteriditis* develop resistance to common antibiotics, treating infections becomes increasingly challenging, leading to higher healthcare costs and greater risks for affected populations. *Salmonella enteriditis*, a significant cause of foodborne illness globally, has exhibited growing resistance to antibiotics, complicating treatment and control efforts.
>
> This project aims to develop a predictive model that forecasts antibiotic resistance trends in *Salmonella enteriditis* by analyzing isolate data. Leveraging machine learning techniques, the model will utilize antibiotic susceptibility testing (AST) results along with specimen metadata, such as the year of collection and geographic region, to identify emerging resistance patterns. These predictions are intended to inform public health strategies, optimize resource allocation, and ultimately improve food safety measures.

## Project Objectives
> - **Forecast Antibiotic Resistance**: Develop a model that predicts resistance patterns in *Salmonella enteriditis* using AST results and metadata, enabling timely interventions.
> - **Support Public Health and Food Safety**: Provide actionable insights to healthcare providers and policymakers that can guide decisions on resource allocation and risk management.
> - **Explore Data-Driven Solutions**: Apply data science techniques, including data wrangling, exploratory data analysis, and machine learning, to address a real-world issue with far-reaching implications.

## Data Source
>The dataset used in this project is sourced from the CDC National Antimicrobial Resistance Monitoring System (NARMS), providing comprehensive surveillance data on antimicrobial resistance in foodborne pathogens. This dataset includes detailed information on antibiotic susceptibility testing and resistance determinants, making it well-suited for predictive modeling of resistance trends.


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
# Load the Excel file
file_path = 'salmonella_enteritidis_isolate_data.csv'  # Replace with your file path
salmonella_isolate_df = pd.read_csv(file_path)

C:\Users\rgarl\AppData\Local\Temp\ipykernel_34208\1496524901.py:3: DtypeWarning: Columns (60,62,108,110) have mixed types. Specify dtype option on import or set low_memory=False.
  salmonella_isolate_df = pd.read_csv(file_path)


In [3]:
# Display the first few rows of the dataset to understand its structure
salmonella_isolate_df.head()

,Specimen ID,Genus,Species,Serotype,Data Year,Region Name,Age Group,Specimen Source,AMI Equiv,AMI Rslt,...,TEL Concl,TEL ConclPred,TET Equiv,TET Rslt,TET Concl,TET ConclPred,TIO Equiv,TIO Rslt,TIO Concl,TIO ConclPred
0,AM15266,Salmonella,enterica,Enteritidis,2003,Region 5,30-39,Stool,=,1.0,...,NaN,NaN,<=,4,S,NaN,=,2.0,S,NaN
1,AM43140,Salmonella,enterica,Enteritidis,2010,Region 2,NaN,Stool,=,1.0,...,NaN,NaN,<=,4,S,NaN,=,1.0,S,NaN
2,AM04769,Salmonella,enterica,Enteritidis,1998,Region 2,30-39,Stool,<=,4.0,...,NaN,NaN,<=,4,S,NaN,<=,0.5,S,NaN
3,AM43324,Salmonella,enterica,Enteritidis,2010,Region 9,40-49,Stool,=,1.0,...,NaN,NaN,<=,4,S,NaN,=,1.0,S,NaN
4,AM29874,Salmonella,enterica,Enteritidis,2006,Region 10,NaN,Stool,=,1.0,...,NaN,NaN,<=,4,S,NaN,=,1.0,S,NaN


In [4]:
# Check for duplicates in the 'Specimen ID' column
duplicate_specimen_ids = salmonella_isolate_df[salmonella_isolate_df.duplicated(subset='Specimen ID', keep=False)]

# Count of duplicate entries
duplicate_count = duplicate_specimen_ids.shape[0]

duplicate_count

0

In [5]:
# Display all columns with 'object' data type
# This helps to identify potential categorical variables for further processing
object_columns = salmonella_isolate_df.select_dtypes(include='object').columns

# Print the names of all object-type columns
print("Object-type columns:", object_columns.tolist())

Object-type columns: ['Specimen ID', 'Genus', 'Species', 'Serotype', 'Region Name', 'Age Group', 'Specimen Source', 'AMI Equiv', 'AMI Concl', 'AMI ConclPred', 'AMP Equiv', 'AMP Concl', 'AMP ConclPred', 'ATM Equiv', 'ATM Concl', 'ATM ConclPred', 'AUG Equiv', 'AUG Concl', 'AUG ConclPred', 'AXO Equiv', 'AXO Concl', 'AXO ConclPred', 'AZM Equiv', 'AZM Concl', 'AZM ConclPred', 'CAZ Equiv', 'CAZ Concl', 'CAZ ConclPred', 'CCV Equiv', 'CCV Concl', 'CCV ConclPred', 'CEP Equiv', 'CEP Concl', 'CEP ConclPred', 'CEQ Equiv', 'CEQ Concl', 'CEQ ConclPred', 'CHL Equiv', 'CHL Concl', 'CHL ConclPred', 'CIP Equiv', 'CIP Concl', 'CIP ConclPred', 'CLI ConclPred', 'COL Equiv', 'COL Concl', 'COL ConclPred', 'COT Equiv', 'COT Concl', 'COT ConclPred', 'CTC Equiv', 'CTC Concl', 'CTC ConclPred', 'CTX Equiv', 'CTX Concl', 'CTX ConclPred', 'ERY ConclPred', 'FEP Equiv', 'FEP Concl', 'FEP ConclPred', 'FFN ConclPred', 'FIS Equiv', 'FIS Concl', 'FIS ConclPred', 'FOX Equiv', 'FOX Concl', 'FOX ConclPred', 'GEN Equiv', 'GE

In [6]:
# Identify columns with all missing values (100% NaN) before removing them
# This step is crucial to eliminate any columns that do not contribute to the analysis
columns_all_missing = salmonella_isolate_df.columns[salmonella_isolate_df.isnull().all()]
columns_all_missing

Index(['CLI Equiv', 'CLI Rslt', 'CLI Concl', 'ERY Equiv', 'ERY Rslt',
       'ERY Concl', 'FFN Equiv', 'FFN Rslt', 'FFN Concl', 'TEL Equiv',
       'TEL Rslt', 'TEL Concl'],
      dtype='object')

In [7]:
# Define the list of columns to be removed 
columns_to_remove = [
    'Genus', 'Species', 'Serotype', 'CLI Equiv', 'CLI Rslt', 'CLI Concl', 'ERY Equiv', 'ERY Rslt',
    'ERY Concl', 'FFN Equiv', 'FFN Rslt', 'FFN Concl', 'TEL Equiv', 'TEL Rslt', 'TEL Concl'
]
# Remove the specified columns from the dataset
salmonella_isolate_df_cleaned = salmonella_isolate_df.drop(columns=columns_to_remove)

In [8]:
# Identify columns to remove based on suffixes
columns_to_remove_suffix = salmonella_isolate_df_cleaned.filter(regex='Equiv$|Rslt$|ConclPred$').columns.tolist()
# Remove the specified columns from the DataFrame
salmonella_isolate_df_cleaned = salmonella_isolate_df_cleaned.drop(columns=columns_to_remove_suffix)

In [9]:
# Count the number of missing values (NaN) in the 'Data Year' column
missing_values_count = salmonella_isolate_df_cleaned['Data Year'].isnull().sum()
missing_values_count

0

In [10]:
salmonella_isolate_df_cleaned['Data Year'].dtype

dtype('int64')

In [11]:
# Investigate counts of each year in the 'Data Year' column
year_counts = salmonella_isolate_df_cleaned['Data Year'].value_counts()

# Sort the counts from low to high
sorted_year_counts = year_counts.sort_index()

# Display the sorted counts
sorted_year_counts

Data Year
1996    354
1997    296
1998    245
1999    266
2000    319
2001    276
2002    337
2003    256
2004    271
2005    384
2006    412
2007    385
2008    442
2009    410
2010    513
2011    391
2012    364
2013    382
2014    438
2015    471
2016    437
2017    431
2018    441
2019    404
2020    280
2021    291
2022    422
2023    339
Name: count, dtype: int64

In [12]:
# Calculate null values in each column and print columns with missing values
null_counts = salmonella_isolate_df_cleaned.isnull().sum()
missing_values = null_counts[null_counts > 0]
missing_values

Age Group            447
Specimen Source       52
AMI Concl           5421
ATM Concl          10220
AZM Concl           5124
CAZ Concl          10220
CCV Concl          10226
CEP Concl           7908
CEQ Concl          10220
COL Concl           8923
CTC Concl          10226
CTX Concl          10220
FEP Concl          10220
FIS Concl           2349
FOX Concl           1111
IMI Concl           9946
KAN Concl           3955
MER Concl           7212
PTZ Concl          10220
SMX Concl           7908
STR Concl           1334
TIO Concl           3045
dtype: int64

In [13]:
# Impute missing values in 'Age Group' and 'Specimen Source' with the mode
# This ensures that missing categorical data is handled appropriately, maintaining the integrity of the dataset
salmonella_isolate_df_cleaned['Age Group'] = salmonella_isolate_df_cleaned['Age Group'].fillna(salmonella_isolate_df_cleaned['Age Group'].mode()[0])
salmonella_isolate_df_cleaned['Specimen Source'] = salmonella_isolate_df_cleaned['Specimen Source'].fillna(salmonella_isolate_df_cleaned['Specimen Source'].mode()[0])

In [14]:
# Drop columns with more than 90% missing values
threshold = 0.9 * len(salmonella_isolate_df_cleaned)
columns_to_drop_90_percent_missing = salmonella_isolate_df_cleaned.columns[salmonella_isolate_df_cleaned.isnull().sum() > threshold]
salmonella_isolate_df_cleaned = salmonella_isolate_df_cleaned.drop(columns=columns_to_drop_90_percent_missing)

In [15]:
# Calculate null values in each column
null_counts = salmonella_isolate_df_cleaned.isnull().sum()

# Filter to show only columns with missing values
missing_values = null_counts[null_counts > 0]

# Print column names and their corresponding missing values
for column, count in missing_values.items():
    print(f"Column: {column}, Missing Values: {count}")

Column: AMI Concl, Missing Values: 5421
Column: AZM Concl, Missing Values: 5124
Column: CEP Concl, Missing Values: 7908
Column: COL Concl, Missing Values: 8923
Column: FIS Concl, Missing Values: 2349
Column: FOX Concl, Missing Values: 1111
Column: KAN Concl, Missing Values: 3955
Column: MER Concl, Missing Values: 7212
Column: SMX Concl, Missing Values: 7908
Column: STR Concl, Missing Values: 1334
Column: TIO Concl, Missing Values: 3045


In [16]:
# Impute missing values in 'Concl' columns with 'X' to indicate missing data
# This signifies that breakpoints or epidemiological cutoff values are not available for certain tests
for col in [col for col in salmonella_isolate_df_cleaned.columns if col.endswith('Concl')]:
    salmonella_isolate_df_cleaned[col] = salmonella_isolate_df_cleaned[col].fillna('X')

In [17]:
# Get all conclusion columns from the cleaned DataFrame
concl_columns = [col for col in salmonella_isolate_df_cleaned.columns if 'Concl' in col]

# Create a copy of the DataFrame to avoid modifying the original
imputed_df = salmonella_isolate_df_cleaned.copy()

# Initialize a dictionary to hold yearly percentages for each column
yearly_percentages = {}

# Loop through each conclusion column to calculate yearly percentages
for col in concl_columns:
    # Calculate the total counts for each category per year
    total_counts = salmonella_isolate_df_cleaned.groupby('Data Year')[col].value_counts(normalize=True).unstack(fill_value=0) * 100
    
    # Store the percentages in the dictionary
    yearly_percentages[col] = total_counts

# Display the calculated yearly percentages for each conclusion column
yearly_percentages

{'AMI Concl': AMI Concl           S           X
 Data Year                        
 1996         7.062147   92.937853
 1997        99.662162    0.337838
 1998       100.000000    0.000000
 1999       100.000000    0.000000
 2000       100.000000    0.000000
 2001       100.000000    0.000000
 2002       100.000000    0.000000
 2003       100.000000    0.000000
 2004       100.000000    0.000000
 2005       100.000000    0.000000
 2006       100.000000    0.000000
 2007       100.000000    0.000000
 2008       100.000000    0.000000
 2009       100.000000    0.000000
 2010       100.000000    0.000000
 2011         0.000000  100.000000
 2012         0.000000  100.000000
 2013         0.000000  100.000000
 2014         0.000000  100.000000
 2015         0.000000  100.000000
 2016         0.000000  100.000000
 2017         0.000000  100.000000
 2018         0.000000  100.000000
 2019         0.000000  100.000000
 2020         0.000000  100.000000
 2021         0.000000  100.000000
 2022  

In [18]:
# Filter the DataFrame for the years 2003 to 2023
salmonella_isolate_df_cleaned = salmonella_isolate_df_cleaned[salmonella_isolate_df_cleaned['Data Year'].between(2003, 2023)]

In [19]:
# Remove the specified columns from the DataFrame
columns_to_remove = [
    'AMI Concl',
    'AZM Concl',
    'CEP Concl',
    'COL Concl',
    'KAN Concl',
    'MER Concl',
    'SMX Concl',
    'TIO Concl'
]

# Drop the specified columns
salmonella_isolate_df_cleaned.drop(columns=columns_to_remove, inplace=True)

In [20]:
# Get all conclusion columns from the cleaned DataFrame
concl_columns = [col for col in salmonella_isolate_df_cleaned.columns if 'Concl' in col]

# Display all conclusion columns
concl_columns

['AMP Concl',
 'AUG Concl',
 'AXO Concl',
 'CHL Concl',
 'CIP Concl',
 'COT Concl',
 'FIS Concl',
 'FOX Concl',
 'GEN Concl',
 'NAL Concl',
 'STR Concl',
 'TET Concl']

In [21]:
# Display the calculated yearly percentages for each conclusion column
yearly_percentages = {}

# Loop through each conclusion column to calculate yearly percentages
for col in concl_columns:
    # Calculate the total counts for each category per year
    total_counts = salmonella_isolate_df_cleaned.groupby('Data Year')[col].value_counts(normalize=True).unstack(fill_value=0) * 100
    
    # Store the percentages in the dictionary
    yearly_percentages[col] = total_counts

# Display the calculated yearly percentages for each conclusion column
yearly_percentages_outputs = {col: data for col, data in yearly_percentages.items()}
yearly_percentages_outputs

{'AMP Concl': AMP Concl         I         R          S
 Data Year                               
 2003       0.000000  2.343750  97.656250
 2004       0.000000  4.059041  95.940959
 2005       0.000000  2.604167  97.395833
 2006       0.000000  4.126214  95.873786
 2007       0.000000  2.077922  97.922078
 2008       0.226244  4.072398  95.701357
 2009       0.000000  3.902439  96.097561
 2010       0.000000  2.339181  97.660819
 2011       0.000000  5.115090  94.884910
 2012       0.000000  4.120879  95.879121
 2013       0.000000  5.759162  94.240838
 2014       0.000000  3.196347  96.803653
 2015       0.000000  5.944798  94.055202
 2016       0.228833  5.263158  94.508009
 2017       0.000000  3.248260  96.751740
 2018       0.000000  1.814059  98.185941
 2019       0.000000  2.722772  97.277228
 2020       0.000000  4.285714  95.714286
 2021       0.000000  3.092784  96.907216
 2022       0.000000  3.080569  96.919431
 2023       0.000000  4.129794  95.870206,
 'AUG Concl': AUG Co

In [22]:
# Count the occurrences of X in the FIS Concl column for the year 2003
x_count_2003 = salmonella_isolate_df_cleaned.loc[
    (salmonella_isolate_df_cleaned['Data Year'] == 2003), 
    'FIS Concl'
].value_counts().get('X', 0)

# Calculate the average percentages for R and S from 2004-2009
average_r = (1.845018 + 1.562500 + 1.456311 + 1.558442 + 1.357466 + 1.707317) / 6
average_s = (98.154982 + 98.437500 + 98.543689 + 98.441558 + 98.642534 + 98.292683) / 6

# Create a function to assign values based on the calculated percentages
def assign_values_for_2003(count):
    # Ensure count is an integer
    count = int(count)  # Convert to integer
    r_count = int(count * (average_r / (average_r + average_s)))  # proportion of R
    s_count = count - r_count  # rest will be S
    
    # Create the list to assign
    return ['R'] * r_count + ['S'] * s_count

# Assign values for 2003
x_values_2003 = assign_values_for_2003(x_count_2003)

# Randomly shuffle the assigned values
np.random.shuffle(x_values_2003)

# Update the DataFrame
salmonella_isolate_df_cleaned.loc[salmonella_isolate_df_cleaned['Data Year'] == 2003, 'FIS Concl'] = x_values_2003

In [23]:
# Display the calculated yearly percentages for each conclusion column
yearly_percentages = {}

# Loop through each conclusion column to calculate yearly percentages
for col in concl_columns:
    # Calculate the total counts for each category per year
    total_counts = salmonella_isolate_df_cleaned.groupby('Data Year')[col].value_counts(normalize=True).unstack(fill_value=0) * 100
    
    # Store the percentages in the dictionary
    yearly_percentages[col] = total_counts

# Display the calculated yearly percentages for each conclusion column
yearly_percentages_outputs = {col: data for col, data in yearly_percentages.items()}
yearly_percentages_outputs

{'AMP Concl': AMP Concl         I         R          S
 Data Year                               
 2003       0.000000  2.343750  97.656250
 2004       0.000000  4.059041  95.940959
 2005       0.000000  2.604167  97.395833
 2006       0.000000  4.126214  95.873786
 2007       0.000000  2.077922  97.922078
 2008       0.226244  4.072398  95.701357
 2009       0.000000  3.902439  96.097561
 2010       0.000000  2.339181  97.660819
 2011       0.000000  5.115090  94.884910
 2012       0.000000  4.120879  95.879121
 2013       0.000000  5.759162  94.240838
 2014       0.000000  3.196347  96.803653
 2015       0.000000  5.944798  94.055202
 2016       0.228833  5.263158  94.508009
 2017       0.000000  3.248260  96.751740
 2018       0.000000  1.814059  98.185941
 2019       0.000000  2.722772  97.277228
 2020       0.000000  4.285714  95.714286
 2021       0.000000  3.092784  96.907216
 2022       0.000000  3.080569  96.919431
 2023       0.000000  4.129794  95.870206,
 'AUG Concl': AUG Co

In [24]:
# Initialize a dictionary to hold yearly percentages for STR Concl
yearly_percentages_str = {}

# Loop through each year to calculate percentages
for year in range(2003, 2024):
    # Filter the data for the specific year
    yearly_data = salmonella_isolate_df_cleaned[salmonella_isolate_df_cleaned['Data Year'] == year]
    
    # Count occurrences of each value in STR Concl
    counts = yearly_data['STR Concl'].value_counts(normalize=True) * 100  # Get percentages
    
    # Store the percentages in the dictionary
    yearly_percentages_str[year] = counts

# Display the calculated yearly percentages for STR Concl
yearly_percentages_str

{2003: STR Concl
 S    98.828125
 R     1.171875
 Name: proportion, dtype: float64,
 2004: STR Concl
 S    97.785978
 R     2.214022
 Name: proportion, dtype: float64,
 2005: STR Concl
 S    98.958333
 R     1.041667
 Name: proportion, dtype: float64,
 2006: STR Concl
 S    98.786408
 R     1.213592
 Name: proportion, dtype: float64,
 2007: STR Concl
 S    99.480519
 R     0.519481
 Name: proportion, dtype: float64,
 2008: STR Concl
 S    99.321267
 R     0.678733
 Name: proportion, dtype: float64,
 2009: STR Concl
 S    98.780488
 R     1.219512
 Name: proportion, dtype: float64,
 2010: STR Concl
 S    99.415205
 R     0.584795
 Name: proportion, dtype: float64,
 2011: STR Concl
 S    98.209719
 R     1.790281
 Name: proportion, dtype: float64,
 2012: STR Concl
 S    98.076923
 R     1.923077
 Name: proportion, dtype: float64,
 2013: STR Concl
 S    97.382199
 R     2.617801
 Name: proportion, dtype: float64,
 2014: STR Concl
 S    97.031963
 R     2.968037
 Name: proportion, dtype: f

In [25]:
# Calculate historical averages for R and S from 2003 to 2019
average_r = sum([yearly_percentages_str[year].get('R', 0) for year in range(2003, 2020)]) / (2020 - 2003)
average_s = sum([yearly_percentages_str[year].get('S', 0) for year in range(2003, 2020)]) / (2020 - 2003)

# Impute missing values for STR Concl from 2020 to 2023
for year in range(2020, 2024):
    # Count total records for the year
    total_counts = salmonella_isolate_df_cleaned[salmonella_isolate_df_cleaned['Data Year'] == year].shape[0]

    # Calculate number of R and S based on the average proportions
    r_count = int(total_counts * (average_r / (average_r + average_s)))
    s_count = total_counts - r_count  # R + S should equal total_counts

    # Update the DataFrame
    salmonella_isolate_df_cleaned.loc[salmonella_isolate_df_cleaned['Data Year'] == year, 'STR Concl'] = 'R'
    salmonella_isolate_df_cleaned.loc[salmonella_isolate_df_cleaned['Data Year'] == year, 'STR Concl'] = 'S'

In [26]:
# Initialize a dictionary to store counts of 'X' values
x_values_count = {}

# Identify all conclusion columns
conclusion_columns = [col for col in salmonella_isolate_df_cleaned.columns if 'Concl' in col]

# Count 'X' values in each conclusion column
for col in conclusion_columns:
    x_count = salmonella_isolate_df_cleaned[col].value_counts().get('X', 0)
    x_values_count[col] = x_count

print(x_values_count)

{'AMP Concl': 0, 'AUG Concl': 0, 'AXO Concl': 0, 'CHL Concl': 0, 'CIP Concl': 0, 'COT Concl': 0, 'FIS Concl': 0, 'FOX Concl': 0, 'GEN Concl': 0, 'NAL Concl': 0, 'STR Concl': 2, 'TET Concl': 0}


In [27]:
# Display the calculated yearly percentages for each conclusion column
yearly_percentages = {}

# Loop through each conclusion column to calculate yearly percentages
for col in concl_columns:
    # Calculate the total counts for each category per year
    total_counts = salmonella_isolate_df_cleaned.groupby('Data Year')[col].value_counts(normalize=True).unstack(fill_value=0) * 100
    
    # Store the percentages in the dictionary
    yearly_percentages[col] = total_counts

# Display the calculated yearly percentages for each conclusion column
yearly_percentages_outputs = {col: data for col, data in yearly_percentages.items()}
yearly_percentages_outputs

{'AMP Concl': AMP Concl         I         R          S
 Data Year                               
 2003       0.000000  2.343750  97.656250
 2004       0.000000  4.059041  95.940959
 2005       0.000000  2.604167  97.395833
 2006       0.000000  4.126214  95.873786
 2007       0.000000  2.077922  97.922078
 2008       0.226244  4.072398  95.701357
 2009       0.000000  3.902439  96.097561
 2010       0.000000  2.339181  97.660819
 2011       0.000000  5.115090  94.884910
 2012       0.000000  4.120879  95.879121
 2013       0.000000  5.759162  94.240838
 2014       0.000000  3.196347  96.803653
 2015       0.000000  5.944798  94.055202
 2016       0.228833  5.263158  94.508009
 2017       0.000000  3.248260  96.751740
 2018       0.000000  1.814059  98.185941
 2019       0.000000  2.722772  97.277228
 2020       0.000000  4.285714  95.714286
 2021       0.000000  3.092784  96.907216
 2022       0.000000  3.080569  96.919431
 2023       0.000000  4.129794  95.870206,
 'AUG Concl': AUG Co

In [28]:
# Check for any remaining 'X' values
remaining_x_count = (salmonella_isolate_df_cleaned['STR Concl'] == 'X').sum()
print(f"Remaining 'X' values in STR Concl: {remaining_x_count}")

Remaining 'X' values in STR Concl: 2


In [29]:
# Drop rows with 'X' values in the STR Concl column
salmonella_isolate_df_cleaned = salmonella_isolate_df_cleaned[salmonella_isolate_df_cleaned['STR Concl'] != 'X']

# Check the shape of the DataFrame after dropping
updated_shape = salmonella_isolate_df_cleaned.shape
print("Updated DataFrame shape:", updated_shape)

Updated DataFrame shape: (8162, 17)


In [30]:
# Check the entire DataFrame for any remaining null values
# This will return the count of null values for each column in the DataFrame
null_values_df = salmonella_isolate_df_cleaned.isnull().sum()

# Display columns that still have null values, if any
null_values_df[null_values_df > 0]

Series([], dtype: int64)

In [31]:
# Assuming salmonella_isolate_df_cleaned is defined and contains the 'Age Group' column
# Print the counts of each age group
age_group_counts = salmonella_isolate_df_cleaned['Age Group'].value_counts()
age_group_counts

Age Group
20-29    1331
50-59    1027
30-39     966
40-49     933
0-4       856
10-19     842
60-69     822
70-79     549
5-9       524
80+       312
Name: count, dtype: int64

In [32]:
# Print the counts of each region in the 'Region' column
region_counts = salmonella_isolate_df_cleaned['Region Name'].value_counts()

# Display the counts
region_counts

Region Name
Region 5     1576
Region 4     1276
Region 3     1152
Region 9     1031
Region 2      981
Region 1      604
Region 6      521
Region 7      412
Region 10     317
Region 8      292
Name: count, dtype: int64

In [33]:
# Print the counts of each region in the 'Region' column
specimen_source_counts = salmonella_isolate_df_cleaned['Specimen Source'].value_counts()

# Display the counts
specimen_source_counts

Specimen Source
Stool           6975
Blood            601
Urine            356
Other            111
Not Given         48
Unknown           40
Wound             15
Abscess           12
CSF                2
Gall Bladder       2
Name: count, dtype: int64

In [34]:
# Rename all entries in 'Specimen Source' that are 'Not Given' to 'Unknown'
salmonella_isolate_df_cleaned['Specimen Source'] = salmonella_isolate_df_cleaned['Specimen Source'].replace('Not Given', 'Unknown')

# Display the updated counts for 'Specimen Source' to verify the change
updated_specimen_counts = salmonella_isolate_df_cleaned['Specimen Source'].value_counts()
updated_specimen_counts

Specimen Source
Stool           6975
Blood            601
Urine            356
Other            111
Unknown           88
Wound             15
Abscess           12
CSF                2
Gall Bladder       2
Name: count, dtype: int64

In [35]:
# Generate summary statistics for object columns
object_summary = salmonella_isolate_df_cleaned.describe(include=['object'])
object_summary

,Specimen ID,Region Name,Age Group,Specimen Source,AMP Concl,AUG Concl,AXO Concl,CHL Concl,CIP Concl,COT Concl,FIS Concl,FOX Concl,GEN Concl,NAL Concl,STR Concl,TET Concl
count,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162,8162
unique,8162,10,10,9,3,3,3,3,3,2,2,3,3,2,2,3
top,AM15266,Region 5,20-29,Stool,S,S,S,S,S,S,S,S,S,S,S,S
freq,1,1576,1331,6975,7859,8096,8139,8078,6962,8108,7993,8115,8145,6976,8020,7854


In [36]:
# Create the 'AR Level' column by counting the number of resistances (R) across all selected antibiotics
# This new feature allows for analysis of multi-antibiotic resistance levels
antibiotic_columns = [col for col in salmonella_isolate_df_cleaned.columns if col.endswith('Concl')]

salmonella_isolate_df_cleaned['AR Level'] = salmonella_isolate_df_cleaned[antibiotic_columns].apply(lambda row: row.str.count('R').sum(), axis=1)

# Sort AR Level counts in ascending order and display
mar_level_counts = salmonella_isolate_df_cleaned['AR Level'].value_counts().sort_index()
mar_level_counts

AR Level
0    6592
1    1271
2     110
3      71
4      73
5      29
6       6
7       4
8       3
9       3
Name: count, dtype: int64

In [37]:
salmonella_isolate_df_cleaned.head()

,Specimen ID,Data Year,Region Name,Age Group,Specimen Source,AMP Concl,AUG Concl,AXO Concl,CHL Concl,CIP Concl,COT Concl,FIS Concl,FOX Concl,GEN Concl,NAL Concl,STR Concl,TET Concl,AR Level
0,AM15266,2003,Region 5,30-39,Stool,S,S,S,I,I,S,S,S,S,R,S,S,1
1,AM43140,2010,Region 2,20-29,Stool,S,S,S,S,S,S,S,S,S,S,S,S,0
3,AM43324,2010,Region 9,40-49,Stool,S,S,S,S,I,S,S,S,S,R,S,S,1
4,AM29874,2006,Region 10,20-29,Stool,R,S,S,S,I,S,R,S,S,R,R,S,4
5,AM18769,2003,Region 2,5-9,Stool,S,S,S,S,S,S,S,S,S,S,S,S,0


In [38]:
# Save the DataFrame as a CSV file
salmonella_isolate_df_cleaned.to_csv('salmonella_isolate_data_cleaned.csv', index=False)

# Confirm that the file has been saved
import os
file_saved = os.path.isfile('salmonella_isolate_data_cleaned.csv')
file_saved

True

In [39]:
# Identify columns that end with 'Concl'
antibiotic_columns = [col for col in salmonella_isolate_df_cleaned.columns if col.endswith('Concl')]

# Group by Data Year and Region Name
grouped_data = salmonella_isolate_df_cleaned.groupby(['Data Year', 'Region Name'])[antibiotic_columns].agg(
    lambda x: (x == 'R').sum() / len(x) * 100
).reset_index()

# Rename columns to indicate percentage
grouped_data.columns = ['Data Year', 'Region Name'] + [f'{col.replace(" Concl", "")} Resistance Percentage' for col in antibiotic_columns]

# Format resistance percentage columns to 2 decimal places
for col in grouped_data.columns[2:]:  # Skip the first two columns (Data Year and Region Name)
    grouped_data[col] = grouped_data[col].round(2)

# Calculate total resistance by summing the resistance percentages
grouped_data['Total Resistance Percentage'] = grouped_data.iloc[:, 2:].sum(axis=1)

In [40]:
grouped_data

,Data Year,Region Name,AMP Resistance Percentage,AUG Resistance Percentage,AXO Resistance Percentage,CHL Resistance Percentage,CIP Resistance Percentage,COT Resistance Percentage,FIS Resistance Percentage,FOX Resistance Percentage,GEN Resistance Percentage,NAL Resistance Percentage,STR Resistance Percentage,TET Resistance Percentage,Total Resistance Percentage
0,2003,Region 1,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,11.11,0.00,0.00,11.11
1,2003,Region 10,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
2,2003,Region 2,6.90,0.0,0.0,0.00,0.0,0.00,3.45,0.0,3.45,0.00,6.90,0.00,20.70
3,2003,Region 3,2.08,0.0,0.0,0.00,0.0,0.00,4.17,0.0,0.00,0.00,0.00,2.08,8.33
4,2003,Region 4,7.69,0.0,0.0,2.56,0.0,2.56,0.00,0.0,0.00,7.69,2.56,2.56,25.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2023,Region 5,3.33,0.0,0.0,0.00,0.0,0.00,1.67,0.0,0.00,36.67,0.00,1.67,43.34
206,2023,Region 6,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,38.10,0.00,0.00,38.10
207,2023,Region 7,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,18.18,0.00,0.00,18.18
208,2023,Region 8,11.76,0.0,0.0,0.00,0.0,0.00,5.88,0.0,0.00,35.29,0.00,0.00,52.93


In [41]:
# Save the new DataFrame to a CSV file
output_file_path = 'salmonella_antibiotic_resistance_percentage.csv'
grouped_data.to_csv(output_file_path, index=False)

## Workflow Overview

> Data Loading: The dataset was loaded from the CDC National Antimicrobial Resistance Monitoring System (NARMS) to ensure reliable and comprehensive surveillance data.
>
> Data Wrangling:
> - Checking for Duplicates: Verified that no duplicates existed in the Specimen ID column, which serves as the unique identifier for each isolate.
> - Initial Exploration: Displayed the first few rows of the dataset to understand its structure and identify potential issues, such as missing values.
Removal of Redundant Columns: Columns deemed unnecessary for analysis, including those with a high proportion of missing values, were identified and removed, streamlining the dataset for further analysis.
Handling Missing Values:
> - Categorical Columns: Missing values in critical categorical columns (e.g., Age Group and Specimen Source) were filled with the mode to maintain data integrity.
Conclusion Columns: Missing entries in conclusion columns were imputed with 'X', indicating that breakpoints or epidemiological cutoff values were unavailable.
> - Excessive Missing Values: Columns with over 90% missing values were removed to focus on relevant features.
> - Data Type Verification: Data types for all columns were checked and ensured to be appropriate for analysis, making necessary conversions for compatibility with modeling techniques.
> - Creation of New Features: A new feature, AR Level, was derived by counting the number of antibiotic resistances across the dataset, providing a quantitative measure of multi-antibiotic resistance.
> - Summary Statistics: Summary statistics were generated to understand the distribution of numeric data, and the data was saved as a cleaned CSV file for future use.
>
> Resistance Calculation: The dataset was further processed to group data by year and region, calculating antibiotic resistance percentages and total resistance levels. This new dataframe was saved as a csv for further processing.
>
>By following these steps, we have prepared a clean, structured dataset ready for exploratory data analysis and predictive modeling, supporting the overall objectives of this project in understanding antibiotic resistance patterns in Salmonella Enteriditis.